## Calculating Transfer Function between two TE accelerometers with a labJack T7-Pro
Uses 9 analog inputs (AINs) to read the data at 1000 Hz.

Craig Lage - Sep 17, 2021


In [ ]:
import sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft, fftfreq, ifft, irfft
from scipy.interpolate import splprep, splrep, splev
from scipy.signal import TransferFunction as tf
from scipy.signal import dlsim, csd, welch
from scipy.optimize import fmin_powell, minimize

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
scanRate = 1000
readTime = 10.0
df = pd.read_pickle("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6.pkl")

In [ ]:
# Plot the data
plt.figure(figsize=(16,8))
plt.subplots_adjust(hspace=0.5)

for n, scanIndex in enumerate([[0, int(scanRate*readTime-1)], [800, 1100], [500,600]]):

    sub_df = df[df.index[scanIndex[0]] : df.index[scanIndex[1]]]
    
    plt.subplot(2,3,n+1)
    ax1 = sub_df['a1z'].plot(label="a1z", color='red')
    ax1.set_title("Axis 1", fontsize=16)
    ax1.set_ylabel("Acceleration(g)")
    ax1.legend(loc='center left')

    plt.subplot(2,3,n+4)
    ax2 = sub_df['a2z'].plot(label="a2z", color='red')
    ax2.set_title("Axis 2", fontsize=16)
    ax2.set_ylabel("Acceleration(g)")
    ax2.legend(loc='center left')
    
    if n == 0: # save this data for the FFT
        a1z = np.array(sub_df['a1z'].tolist())
        a2z = np.array(sub_df['a2z'].tolist())

#plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_14Sep21.pdf")

In [ ]:
# Do the FFT
N = len(a1z)
xAxis = fftfreq(N, 1 / scanRate)
yf1 = fft(a1z)
yf2 = fft(a2z)
print(N, yf1[20], yf2[20])

In [ ]:
plt.title("FT", fontsize=24)
plt.plot(xAxis, np.log10(yf1), marker='x', color='red', label = 'Driven FFT')
plt.plot(xAxis, np.log10(yf2), marker='x', color='green', label = 'Monitor FFT')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)

In [ ]:
# Invert
a1z_inv = ifft(yf1)
N = len(a1z_inv)
print(N, a1z_inv[20])

In [ ]:
plt.title("FT Inverse", fontsize=24)
plt.plot(a1z,  color='red', label = 'Original Signal')
plt.plot(a1z_inv,  color='green', label = 'ifft(fft(Signal))')
#plt.xlabel("Frequency(Hz)", fontsize=16)
#plt.ylabel("X(k)", fontsize=16)

In [ ]:
# Do the FFT - Welch
# How do we invert this?
welchSD1 = welch(a1z, fs=1000)
welchSD2 = welch(a2z, fs=1000)
N = len(welchSD1[0])
print(N, welchSD1[1][20], welchSD2[1][20])

In [ ]:
plt.title("FT", fontsize=24)
plt.plot(welchSD1[0], np.log10(np.sqrt(welchSD1[1])), marker='x', color='red', label = 'FFT')
plt.plot(welchSD1[0], np.log10(np.sqrt(welchSD2[1])), marker='x', color='green', label = 'FFT')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)

In [ ]:




tf = np.sqrt(welchSD2[1]) / np.sqrt(welchSD1[1])
tck, u = splprep([welchSD1[0],tf],u=welchSD1[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("TF", fontsize=24)
plt.plot(welchSD1[0], tf, marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)

In [ ]:
crossSpectralDensity = csd(a1z, a2z, fs=1000)

welchSpectralDensity1 = welch(a1z, fs=1000)
welchSpectralDensity2 = welch(a2z, fs=1000)
#tf =  welchSpectralDensity2[1] / crossSpectralDensity[1] # H2
tf =  csd(a2z, a2z, fs=1000)[1] / csd(a1z, a2z, fs=1000)[1] # H2 Pyy / Pxy
#tf =  csd(a2z, a1z, fs=1000)[1] / csd(a1z, a1z, fs=1000)[1] # H1 doesn't work??
#tf =  np.conj(crossSpectralDensity[1]) / welchSpectralDensity1[1] # H1 doesn't work??
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
tck, u = splprep([welchSpectralDensity1[0],np.real(tf)],u=welchSpectralDensity1[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("TF", fontsize=24)
plt.plot(welchSpectralDensity1[0], tf, marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)
plt.subplot(1,2,2)
tck, u = splprep([welchSpectralDensity1[0],np.imag(tf)],u=welchSpectralDensity1[0],s=20,k=3)
fFit, tfFit = splev(u,tck)
plt.title("TF", fontsize=24)
plt.plot(welchSpectralDensity1[0], np.imag(tf), marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)
plt.xlim(20,200)

In [ ]:
tf[20]

In [ ]:
tf = np.sqrt(welchSD2[1]) / np.sqrt(welchSD1[1])
tck, u = splprep([welchSD1[0],tf],u=welchSD1[0],s=2,k=3)
fFit, tfFit = splev(u,tck)
plt.title("TF", fontsize=24)
plt.plot(welchSD1[0], tf, marker='x', color='blue', label = 'TF')
plt.plot(fFit, tfFit, marker='x', color='red', label = 'Spline')
plt.xlabel("Frequency(Hz)", fontsize=16)
plt.ylabel("X(k)", fontsize=16)

In [ ]:
def calcTransferFunction(scanRate, drivenSeries, monitorSeries):#, orderB=1, orderA=1):
    # Calculates and plots a transfer function given a driven series of data
    # and a monitor series of data

    # Do the FFT
    N = len(drivenSeries)
    xAxis = fftfreq(N, 1 / scanRate)[10:int(N/2-20)]
    yf1 = fft(drivenSeries)[10:int(N/2-20)]
    yf2 = fft(monitorSeries)[10:int(N/2-20)]
    N = len(xAxis)
    # Now plot it

    rmsRatio = np.std(monitorSeries) / np.std(drivenSeries)
    fftPeakRatio = np.max(np.abs(yf2[50:-1])) / np.max(np.abs(yf1[50:-1]))
    #fftPeakRatio = np.max(np.abs(yf2)) / np.max(np.abs(yf1))

    plt.figure(figsize=(16,8))

    # Calculate the transfer function vs frequency and a spline fit and plot them
    crossSpectralDensity = csd(a1z, a2z, fs=1000)
    # H1
    welchSpectralDensity1 = welch(a1z, fs=1000)
    welchSpectralDensity2 = welch(a2z, fs=1000)
    transferFunction = welchSpectralDensity2[1] / welchSpectralDensity1[1]
    
    
    print(welchSpectralDensity1[0][0:5])
    print(welchSpectralDensity1[1][0:5])
    # H2
    #welchSpectralDensity = welch(a2z, fs=1000)
    #transferFunction = welchSpectralDensity[1] / np.conj(crossSpectralDensity[1])
    nKnots = 10
    tck, u = splprep([welchSpectralDensity1[0],transferFunction],u=welchSpectralDensity1[0],s=200,k=3)
    fFit, tfFit = splev(u,tck)
    plt.subplot(1,2,2)
    plt.title("Transfer Function", fontsize=24)
    plt.plot(crossSpectralDensity[0], transferFunction, marker='x', color='blue', label = 'Raw Transfer Function')
    #plt.plot(fFit, tfFit, lw=5.0, color='red', label = 'Spline Fit')
    plt.xlabel("Frequency(Hz)", fontsize=16)
    plt.ylabel("Transfer Function", fontsize=16)
    #plt.xlim(40,400)
    #plt.ylim(0,5)
    plt.legend(fontsize=12)

    fFit2, tfFit2 = splev(xAxis,tck)
    #print(xAxis.min(), xAxis.max(), u.min(), u.max())
    print(len(welchSpectralDensity1[0]), len(u), len(xAxis), len(fFit2), len(tfFit2), N)
    #print(crossSpectralDensity[0][0:5],u[0:5], xAxis[0:5], fFit2[0:5], (fFit2/xAxis)[0:5])
    tfFit2_trimmed = np.clip(tfFit2, 0.0, 100.0)
    yfModel = welchSpectralDensity1[1] * transferFunction
    
    az1_tSpace = irfft(welchSpectralDensity1[1])
    
    plt.subplot(1,2,1)
    plt.subplots_adjust(hspace=0.5)
    #plt.plot(xAxis, np.abs(yf1), color='red', marker = 'x', label='A1Z - Driver')
    plt.plot(welchSpectralDensity1[0], welchSpectralDensity1[1], color='red', marker = 'x', label='A1Z - Driver')
    plt.plot(welchSpectralDensity2[0], welchSpectralDensity2[1], color='green', marker = 'x', label='A2Z - Monitor')
    #plt.plot(xAxis, np.abs(yf2), color='green', marker = 'x', label='A2Z - Monitor')
    plt.plot(welchSpectralDensity2[0], yfModel, color='blue', marker = 'x', label='A2Z - Model')
    plt.title("FFT of accelerometer force transfer", fontsize=24)
    plt.xlabel("Frequency (Hz)",fontsize=16)
    #plt.text(20, 20, "RMS_Ratio = %.3f"%rmsRatio, fontsize=16)
    #plt.text(20, 17, "FFT Peak Ratio = %.3f"%fftPeakRatio, fontsize=16)
    #plt.ylim(0.0,1.0)
    plt.xlim(0,500)
    plt.legend(loc = 'upper left', fontsize=12)

    #plt.savefig("/Users/cslage/Research/LSST/code/labJack/accel_data/Transfer_Test_6_Plot_14Sep21.pdf")
    
    return az1_tSpace

In [ ]:
outSim = calcTransferFunction(scanRate, a1z, a2z)

In [ ]:
# Close handles
ljm.close(handle)

In [ ]:
plt.plot(outSim)
